# Importando librerias

In [1]:
import pandas as pd
import re
import unicodedata
from nltk.corpus import stopwords
import nltk
import time
#import swifter
import numpy as np


# Laod dataset - select `df_sm`

In [2]:
df_sm = pd.read_csv('../data/dev/df_sm_tweets_26072020.csv',encoding='utf8')

In [3]:
df_sm.groupby(['sentiment']).size()

sentiment
NEGATIVE      9479
NEUTRAL     109866
POSITIVE     10976
dtype: int64

# Remove `links`

In [4]:
def remove_links(text):
    text = re.sub(r'www\\S+',r'',text,flags=re.MULTILINE)
    text = re.sub(r'https\S+',r'',text,flags=re.MULTILINE)
    
    return(text)


## Remove `hashtags` and `mentions of person` or `page`

In [5]:
def remove_hashtags_mentions(text):
    #return(' '.join(re.sub("(@|#)(\w+)|[_]+\w+","",text,flags=re.MULTILINE).split()))
    return(' '.join(re.sub("(@|#|&)(\w+)|[_]+\w+","",text,flags=re.MULTILINE).split()))
    

# Replace `emoticons`

In [6]:
SMILEYS = {":‑(":"triste", ":‑)":"feliz"}
## por ahora solo los elimanaremos

# Convert `unicode` to `Ascii`

In [7]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD',s)
        if unicodedata.category(c)!='Mn'
    )

In [8]:

#df_sm['tweet_text'].apply(lambda x: unicodeToAscii(x))

# Preprocesamiento - `stopwords` con _nltk_

In [9]:
#'mas','si','mio','mia','mios','mias', estas
stopwords.words('spanish')
exclude_stopword = ['si',
'sí',
'no',
'más',
'pero',
'ya',
'porque',
'muy',
'sin',
'tambien',
'todo',
'nos',
'ni',
'contra',
'tanto',
'nada',
'poco',
'fue',
'fui',
'fuiste',
'fuimos',
'sentida',
'sentido']


In [10]:
#nlk_stopword
nltk_stopwords = stopwords.words('spanish')
print("Num. exclude stopwords: ",len(exclude_stopword))
print("Num. stopwords",len(nltk_stopwords))


Num. exclude stopwords:  23
Num. stopwords 313


In [11]:
nltk_sw = [e for e in nltk_stopwords if (e not in exclude_stopword)]

In [12]:
nltk_sw[:10]

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']

## Remove stopwords

In [13]:
def remove_stopword(text,list_stopwords):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [token for token in tokens if not token in nltk_sw]
    return ' '.join(tokens)

In [14]:

remove_stopword('hola amigos, tenga su buenos dias',nltk_sw)

'hola amigos , buenos dias'

# Remove `punctuations`

In [15]:
def remove_punctuations(text):
    return ( ' '.join(re.sub("[\.\,\:\;\-\=\_\!\?]",' ',text,flags=re.MULTILINE).split()) )

In [16]:
remove_punctuations('Hola es ESdrújula!'.lower())

'hola es esdrújula'

# Remove - onomatopeyas

In [65]:
def remove_closures(text):
    tokens = str(text).lower().split(' ')
    tokens = [w for w in tokens if not re.search('^[ja]+$', w)]
    tokens = [w for w in tokens if not re.search('^[ji]+$', w)]
    tokens = [w for w in tokens if not re.search('^[je]+$', w)]
    tokens = [w for w in tokens if not re.search('^[ha]+$', w)]
    tokens = [w for w in tokens if not re.search('^[mo]+$', w)]
    #tokens = [w for w in tokens if not re.search('^[jka]+$', w)]
    return(' '.join(tokens))

In [66]:
remove_closures('@Oliver__Cortez @ChelseaFC JAJAJWJQJAJQ LE CREES UNA PALABRA AL AS ???'.split(' '))

"['@oliver__cortez', '@chelseafc', 'jajajwjqjajq', 'le', 'crees', 'una', 'palabra', 'al', 'as', '???']"

In [48]:
remove_closures(' '.join(['jejeje','jajajak','jajaja','jaa','aja']))

'jajajak'

# convert - `stemming`

In [49]:
snowball_stemmer = nltk.stem.SnowballStemmer('spanish')

In [50]:
def convert_stemm(text,snowball_stemmer=nltk.stem.SnowballStemmer('spanish')):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [snowball_stemmer.stem(token) for token in tokens]   
    
    return(' '.join(tokens))
    


In [51]:
def remove_single_letters(text, min_len=1):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [w for w in tokens if len(w)>min_len]
    return(' '.join(tokens))

In [52]:
def remove_long_letters(text, max_len=16):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [w for w in tokens if len(w)<max_len]
    return(' '.join(tokens))

In [53]:
remove_long_letters('hola amigos a m s donde es a los')

'hola amigos a m s donde es a los'

# Normalization - `Pre Procesamiento`

In [54]:
def clean_string(s):
    s = remove_links(s)
    s = remove_hashtags_mentions(s)
    
    s = unicodeToAscii(str(s).lower().strip())
    #s = remove_closures(s)
    
    s = remove_punctuations(s)
    
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s) ##----sin tildes
    s = remove_stopword(s,nltk_sw)
    s = convert_stemm(s)
    
    s = remove_closures(s)
    
    s = remove_single_letters(s) 
    s = remove_long_letters(s)
    s = re.sub(r'(.)\1+', r'\1\1', s) ## --- eliminina repiriticones de 3 o mas veces
    s = re.sub(r"([.!?])", r" \1 ",s)
    return(s)

In [55]:
clean_string('JAJAJA  adJA ')

'adja'

# Aplicando la normalización

In [56]:
%%time
df_sm['tweet_text_norm']= df_sm['tweet_text'].apply(lambda x: clean_string(x))

Wall time: 2min 56s


# export dataset - now

In [60]:
clean_string('@Oliver__Cortez @ChelseaFC JAJAJWJQJAJQ LE CREES UNA PALABRA AL AS ???')

'jajajwjqjajq cre palabr as'

In [57]:
df_sm.to_csv('../data/dev/df_sm_norm_'+'_'.join((re.sub(':','-',time.ctime())).split(' ')[1:])+'.csv',index=None)

In [ ]:
%%time
df_sm['len_words'] = df_sm['tweet_text_norm'].apply(lambda x: len(x.split(' ')))

In [ ]:
np.sort(df_sm['len_words'].unique())

In [ ]:
convert_stemm('funciona')